In [1]:
import tensorflow as tf
from tensorflow.python.keras.datasets import mnist

# input image dimensions
img_rows, img_cols = 28, 28
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_images = 9
some_images = x_test[0:num_images]
some_images_pred = y_test[0:num_images]

In [18]:
import json

for images in some_images.tolist():
    jsondata = {"x_input": images}
    with open('test.json', 'a') as outfile:
        json.dump(jsondata, outfile)
        outfile.write('\n')

In [19]:
!gcloud ml-engine predict \
--model=est_mnist --version=v2 \
--json-instances=./test.json

PROBABILITIES
[3.6881393494780923e-09, 8.658238570546928e-10, 9.342448379356938e-07, 2.28361159315682e-06, 2.0509602696727747e-11, 8.327801781504718e-10, 6.197841719013386e-13, 0.9999966621398926, 1.746771061128527e-10, 6.301718968870773e-08]
[5.293816684570629e-06, 0.0024341249372810125, 0.9975582361221313, 8.728165425964107e-07, 5.537302616431816e-09, 2.1495951685324144e-09, 3.732504012532445e-07, 3.593947894842131e-08, 8.984150667856738e-07, 3.393597625134248e-10]
[6.040726020728471e-06, 0.9998031258583069, 2.502256165826111e-06, 9.803144394027186e-07, 1.4385051144927274e-05, 2.1688208562409272e-06, 2.416542656646925e-06, 0.00015805767907295376, 6.985506388446083e-06, 3.4354645777057158e-06]
[0.9999489784240723, 1.2105174640453242e-08, 3.785765159136645e-07, 5.6825296468332454e-08, 8.116201932750755e-09, 2.812283810271765e-06, 4.574328340822831e-05, 6.637182536906039e-07, 3.4140990123887605e-07, 9.492666208643641e-07]
[1.3575734669757367e-07, 1.4732076181189768e-07, 3.69764393326477

In [21]:
!gsutil cp test.json gs://mnist-estimator/batchpred/inputs.json

Copying file://test.json [Content-Type=application/json]...
\ [1 files][ 66.1 KiB/ 66.1 KiB]                                                
Operation completed over 1 objects/66.1 KiB.                                     


In [22]:
!gcloud ml-engine jobs submit prediction mnist_estimator_pred_`date +%s` \
--region europe-west1 \
--data-format=TEXT \
--input-paths=gs://mnist-estimator/batchpred/inputs.json \
--output-path=gs://mnist-estimator/batchpred/outputs \
--model=est_mnist --version=v2

Job [mnist_estimator_pred_1539596022] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe mnist_estimator_pred_1539596022

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs mnist_estimator_pred_1539596022
jobId: mnist_estimator_pred_1539596022
state: QUEUED
